In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
dbutils.fs.ls("abfss://landing@salesmarketingstorage1.dfs.core.windows.net")

In [0]:
df = spark.read \
.options(header=True) \
.csv("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/customers")

df.show()

In [0]:
df_customers = (
    spark.readStream 
    .format("cloudFiles").option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("quote", "\"")
    .option("mode", "PERMISSIVE")
    .option("cloudFiles.schemaLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/bronze/customers")
    .load("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/customers/")
    .writeStream
    .format("delta")
    .trigger(availableNow=True)
    .option("checkpointLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/bronze/customers") 
    .table("bronze.customers")
)

In [0]:
%sql
SELECT * FROM cloud_files_state('abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/bronze/customers');

In [0]:
for stream in spark.streams.active:
    print(stream.name, stream.isActive, stream.lastProgress)

In [0]:
%sql 
SELECT * FROM bronze.customers
limit 10;

In [0]:
%sql
select * from bronze.customers limit 10;